In [1]:
#explore geo data in the /mnt/storage7//jody/tb_data_2022_08_19.csv
#created - 26.08.2022

from re import sub
from unittest import result
from icecream import ic
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import re
import os
import json
import subprocess
import itertools
from statistics import mean
import pandas as pd
import chart_studio.plotly as py
import plotly.offline as po
import plotly.express as px
import plotly.graph_objs as pg
import matplotlib.pyplot as plt
import pycountry_convert
%matplotlib inline
po.init_notebook_mode(connected = True)
from urllib.request import urlopen
import pycountry
from sklearn import preprocessing

import geopandas as gpd
import geoplot as gplt
import geoplot.crs as gcrs
from matplotlib.colors import LinearSegmentedColormap
import pandas as pd


# plotly-geo
# geopandas --upgrade
# shapely
# python_version_tuple
# chart_studio

In [38]:
#load and remove nan
csv_ = "/mnt/storage7//jody/tb_data_2022_08_19.csv"
df = pd.read_csv(csv_)
df=df.dropna(subset=['country_code', 'geographic_source', 'drug_resistance_type', 'sublineage']).reset_index(drop=True)

/mnt/storage7/lwang/miniconda3/envs/ml-s7/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning:

Columns (12,15,16,17,19,20,47,48,49,51,53,54,56,57,58,59,62,63) have mixed types.Specify dtype option on import or set low_memory=False.



In [49]:
df_c = df_country[df_country["geographic_source"]==x]
df_c.groupby(["drug_resistance_type"])

KeyError: 'geographic_source'

In [53]:
df_country = df.groupby(["geographic_source"]).count()
df_country['geographic_source'] = df_country.index

df_c = df_country[df_country["geographic_source"]=="Algeria"]
df_c.groupby(["drug_resistance_type"])

In [61]:
df["drug_resistance_type"].unique()

'MDR-TB'

In [44]:
# df_country = df.groupby(["geographic_source"]).count()
# df_country['geographic_source'] = df_country.index

# for x in df_country["geographic_source"]:
#     df_c = df_country[df_country["geographic_source"]==x]
#     df_c.groupby(["drug_resistance_type"])

<h2>Creating 3 letter country code for graphing

In [67]:
#converting 2 letter country code to 3 letter
def country_code_conv(two_letter):
    name = pycountry_convert.country_alpha2_to_country_name(two_letter.upper(), cn_name_format="default")
    three_letter = pycountry_convert.country_name_to_country_alpha3(name, cn_name_format="default")

    return three_letter

df['country_3l_code'] = None
for i, e in enumerate(df['country_code']):
    df['country_3l_code'][i] =  country_code_conv(e)

/tmp/ipykernel_19058/2264284554.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [7]:
# # turning drug resistance type into number
# le = preprocessing.LabelEncoder()
# le.fit([df["drug_resistance_type"]])

# drug_resistance_type = le.transform(df["drug_resistance_type"])
# # le.inverse_transform([0, 1, 2, 3, 4, 5])


<h2>Mixed infection

In [9]:
# create a column that indicate whether the samle has mixed infection
df['mixed_infection'] = "None"

for i, x in enumerate(df['sublineage']):
    len_ = len(str(x).split(";"))
    if len_ > 1 and len_ < 3:
        df['mixed_infection'][i] = "Mixed_infection"
    else:
        df['mixed_infection'][i] = "Non-mixed_infection"


df_no_mix = df.loc[df["mixed_infection"]=="Non-mixed_infection"]
df_no_mix = df_no_mix.groupby(["country_3l_code"]).count()
df_no_mix['country_3l_code'] = df_no_mix.index

df_count = df.groupby(["country_3l_code"]).count()
df_count['country_3l_code'] = df_count.index

df_mix = df.loc[df["mixed_infection"]=="Mixed_infection"]
df_mix = df_mix.groupby(["country_3l_code"]).count()
df_mix['country_3l_code'] = df_mix.index

list_ = []
threshold = 10
for i, x in enumerate(df_count["country_3l_code"]):
    if x in df_mix["country_3l_code"]:
        mix = df_mix[df_mix['country_3l_code']==x]["mixed_infection"]
        all = df_count[df_count['country_3l_code']==x]["mixed_infection"]
        if int(all) <= threshold:
            list.append(0)
            continue
        ratio = int(mix) / int(all)
        list_.append(ratio)
    else:
        list_.append(0)

df_count["mix_rate"] = list_


/tmp/ipykernel_19058/300186783.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_19058/300186783.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [35]:
df = px.data.tips()
fig = px.histogram(df_count, x="mixed_infection", nbins=200, title='Histogram of number of infection recorded',labels={'mixed_infection':'infection sample number'})
fig.show()


In [27]:
fig = px.choropleth(df_count, color='mix_rate',
                    locations='country_3l_code',
                   )
fig.update_geos(fitbounds='locations', visible=False)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

<h2>Drug resistance type

In [ ]:
df["drug_resistance_type"].unique()
for x in df["drug_resistance_type"].unique():
    df_dr = df[df["drug_resistance_type"]== x].groupby(["country_3l_code"])
    


In [ ]:
dict_ = {}
for x in df["drug_resistance_type"].unique()
    dict_[x] = []
    df_dr = df[df["drug_resistance_type"]== x].groupby(["country_3l_code"])


        if x in df_mix["country_3l_code"]:

    for y in df_count["country_3l_code"]:




In [36]:
list_ = []
threshold = 10
for i, x in enumerate(df_count["country_3l_code"]):
    if x in df_mix["country_3l_code"]:
        mix = df_mix[df_mix['country_3l_code']==x]["mixed_infection"]
        all = df_count[df_count['country_3l_code']==x]["mixed_infection"]
        if int(all) <= threshold:
            list.append(0)
            continue
        ratio = int(mix) / int(all)
        list_.append(ratio)
    else:
        list_.append(0)

df_count["mix_rate"] = list_

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2


In [ ]:
fig = px.choropleth(df, color='mixed_infection',
                    locations='country_3l_code',
                   )
fig.update_geos(fitbounds='locations', visible=False)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
fig = px.choropleth(df, color='drug_resistance_type',
                    locations='country_3l_code',
                   )
fig.update_geos(fitbounds='locations', visible=False)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
#graphing the geo data
data = dict(type='choropleth', 
            locations = df['country_3l_code'] , 
            # z = df['drug_resistance_type_label'], 
            featureidkey = df['drug_resistance_type'],
            # marker = df["mixed_infection"],
            text = (df['geographic_source'],df['drug_resistance_type']))


layout = dict(title = 'TB-resistance',
            
              geo = dict(showframe = True,
                        projection = {'type':'natural earth'},
                         showlakes = False, 
                         lakecolor = 'rgb(0,191,255)'))


x = pg.Figure(data = [data] ,
layout = layout)
po.iplot(x)

